In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok transformers torch

In [ ]:
!pip install datasets

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset
import numpy as np
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import shutil
import pandas as pd

In [ ]:
# Montée de Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Chargement du fichier CSV dans un DataFrame
df = pd.read_csv('/content/drive/My Drive/projet_7/cleaned_tweets.csv', encoding='latin1', sep=";")

In [ ]:
with open('/content/drive/My Drive/projet_7/token.txt', 'r') as file:
    hf_token = file.read().strip()

from huggingface_hub import login
login(hf_token)

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2o7fRSSTsKkRHY1jl4VoX9qS7AR_5TCYPvixQ8rv7g5PqJp8t")

In [ ]:
# Supprimer les lignes où `text_cleaned` est NaN
df_cleaned = df.dropna(subset=['text_cleaned']).reset_index(drop=True)

# Vérifier les dimensions après suppression
print(f"Nombre de lignes après nettoyage : {df_cleaned.shape[0]}")

# Re-diviser les colonnes
tweets = df_cleaned['text_cleaned'].tolist()
labels = df_cleaned['sentiment'].tolist()

df['text_cleaned'] = df['text_cleaned'].astype(str)  # Convertir en chaînes
df['sentiment'] = df['sentiment'].astype(int)  # Convertir en entiers

Nombre de lignes après nettoyage : 1573595


In [ ]:
df_cleaned_sample = df_cleaned.sample(n=10000, random_state=42)

# Séparer les textes et les labels
tweets = df_cleaned_sample['text_cleaned'].tolist()  # Colonne des tweets
labels = df_cleaned_sample['sentiment'].tolist()  # Colonne des labels (0 ou 1)

# Diviser les données en train/val/test
train_texts, temp_texts, train_labels, temp_labels = train_test_split(
    tweets, labels, test_size=0.3, random_state=42, stratify=labels
)
val_texts, test_texts, val_labels, test_labels = train_test_split(
    temp_texts, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

# Tokenisation
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Créer des datasets Hugging Face
train_data = Dataset.from_dict({"text": train_texts, "label": train_labels})
val_data = Dataset.from_dict({"text": val_texts, "label": val_labels})

train_data = train_data.map(tokenize_function, batched=True)
val_data = val_data.map(tokenize_function, batched=True)

# Charger le modèle
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    report_to="none",  # Pour éviter les problèmes de connexion avec Weights & Biases
    seed=42
)

# Trainer avec Early Stopping
from transformers import EarlyStoppingCallback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Entraîner le modèle
trainer.train()

# Évaluer le modèle
trainer.evaluate()

# Sauvegarder le modèle
model.save_pretrained("./fine_tuned_distilbert")
tokenizer.save_pretrained("./fine_tuned_distilbert")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-2c3377e06faa>:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
